In [1]:
from crewai import Agent, Task, Crew
import os
from dotenv import load_dotenv
import openai
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

In [2]:
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("OPENAI_MODEL_NAME", "gpt-4-turbo")  # Define um padrão caso não esteja no .env

# Configuração do modelo de IA
llm = ChatOpenAI(model_name=model_name, openai_api_key=api_key, temperature=0.3)

# Memória resumida para manter o contexto sem gastar muitos tokens
memory = ConversationSummaryMemory(llm=llm)

C:\Users\guoli\AppData\Local\Temp\ipykernel_37208\1269163713.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name=model_name, openai_api_key=api_key, temperature=0.3)
C:\Users\guoli\AppData\Local\Temp\ipykernel_37208\1269163713.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


In [3]:
Arandom = Agent(
    role="Gerador de conceitos aleatorias",
    goal="Sua tarefa será falar palavras aleatórias, como animais, pessoas, paises, objetos, etc",
    backstory=f"Você deve gerar palavras aleatórias para que o contador de historias possa adicionar em sua historia",
    llm=llm,
    memory=memory
)

Ahistory = Agent(
    role="Contador de historas",
    goal="Sua tarefa será ouvir as palavras aleatórias do gerador de conceitos e criar histórias a partir deles, adicionando os elementos todos a uma mesma história",
    backstory="Você cria histórias, e a partir de conceitos aleatórios deve criar e adicionar esses conceitos em suas histórias",
    llm=llm,
    memory=memory
)

def GetHistoryAIResponse(InputText):
    taskHisotry = Task(
    description=(
        f"Você deve adicionar o seguinte contexto em sua história: {InputText}"
    ),
    agent=Ahistory, 
    memory=memory,
    expected_output="Gere uma história somando os contextos apresentados a sua história, caso ainda não haja história comece com o contexto apresentado" 
    )

    crewBuyer = Crew(
    agents=[Ahistory],
    tasks=[taskHisotry]
    )

    return crewBuyer.kickoff()

def GetRandomAIResponse():
    taskRandom = Task(
    description=(
        "Retorne uma palavra completamente aleatoria, podendo ser um pais, animal, objeto ou qualquer outra coisa, nunca retorne a mesma palavra duas vezes"
    ),
    agent=Arandom, 
    memory=memory,
    expected_output="Apenas uma palavra aleatória"
    )

    crewSeller = Crew(
    agents=[Arandom],
    tasks=[taskRandom]
    )

    return crewSeller.kickoff()

In [ ]:
# Carregar variáveis do .env
load_dotenv()

# Usar as variáveis carregadas
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("OPENAI_MODEL_NAME", "gpt-4o-mini")  # Define um padrão caso não esteja no .env

# Configuração do modelo de IA
llm = ChatOpenAI(model_name=model_name, openai_api_key=api_key, temperature=0.3)

# Simulação do diálogo
RandomSpeaking = True

for i in range(6):
    if RandomSpeaking:
        resultado = GetRandomAIResponse()
        print(resultado)
        RandomSpeaking = False
    else:
        resultado = GetHistoryAIResponse(resultado)
        print(resultado)
        RandomSpeaking = True

Overriding of current TracerProvider is not allowed


Girafa


Overriding of current TracerProvider is not allowed


Em uma vasta savana africana, onde o sol brilhava intensamente e as árvores se erguiam como torres verdes, vivia uma girafa chamada Gigi. Gigi era uma girafa curiosa e sonhadora, sempre em busca de novas aventuras. Um dia, enquanto caminhava entre as acácias, ela ouviu um rumor vindo de um grupo de animais reunidos em um círculo. Intrigada, Gigi se aproximou e viu que todos estavam falando sobre um misterioso lago que, segundo rumores, tinha o poder de realizar desejos.

Movida pela curiosidade, Gigi decidiu que precisava encontrar esse lago mágico. Com seu longo pescoço, ela conseguia avistar as paisagens ao longe e, após horas de caminhada, avistou uma bela água cristalina refletindo o céu azul. Ao chegar ao lago, Gigi ficou maravilhada com a beleza do lugar. As flores ao redor dançavam com a brisa e o som da água era como uma melodia suave.

Gigi se aproximou da margem e, olhando para a água, fez um pedido sincero: "Eu desejo entender melhor os sentimentos dos meus amigos animais." 

Overriding of current TracerProvider is not allowed


...

Overriding of current TracerProvider is not allowed


Em uma remota e gelada região da Antártica, havia um pinguim chamado Pip. Pip era um pinguim diferente dos outros; enquanto seus amigos passavam o dia deslizando pelo gelo e mergulhando nas águas frias em busca de peixes, ele sonhava em explorar o mundo além do horizonte branco. Ele sempre olhava para o céu e imaginava como seria voar como as aves que passavam por ali.

Um dia, enquanto Pip estava em uma de suas reflexões, ele encontrou uma garrafa flutuando no mar. Curioso, ele a puxou para perto e, para sua surpresa, encontrou um mapa dentro dela. O mapa mostrava uma ilha misteriosa, repleta de tesouros e criaturas fantásticas. O coração de Pip disparou de emoção. Ele decidiu que precisava ir até lá, mesmo que isso significasse deixar sua colônia de pinguins.

Com a ajuda de seus amigos, Pip começou a planejar a jornada. Eles construíram uma pequena jangada com pedaços de gelo e madeira que encontraram na praia. No dia da partida, Pip se despediu de sua família e amigos, prometendo v

Overriding of current TracerProvider is not allowed


Abacaxi


KeyboardInterrupt: 

# Tentativa reiterando a ideia inicial

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory

In [6]:
# 1) Configura LLM e memória
llm = ChatOpenAI(openai_api_key=api_key, model_name=model_name, temperature=0.3)
memory = ConversationSummaryMemory(llm=llm)

In [9]:
# 2) Função que gera resposta para um “papel” (comprador ou vendedor, ou gerador de palavra/historiador)
def gerar_resposta(role, objetivo, entrada_outro, memory, llm):
    # Carrega o resumo atual
    resumo = memory.load_memory_variables({})["history"]
    print(resumo)
    # Monta prompt
    prompt = (
        f"Você é {role}.\n"
        f"Seu objetivo: {objetivo}\n\n"
        f"Resumo da conversa até agora:\n{resumo}\n\n"
    )
    if entrada_outro:
        prompt += f"O outro disse: {entrada_outro}\n"
    prompt += "Responda agora:\n"

    # Gera resposta
    resposta = llm.predict(prompt)

    # Salva no histórico
    memory.save_context({"input": entrada_outro or ""}, {"output": resposta})

    return resposta

In [10]:
# 3) Loop de turnos
n_vezes = 6
mensagem_atual = ""
for i in range(n_vezes):
    if i % 2 == 0:
        # Gerador de palavra
        role = "Gerador de Palavras Aleatórias"
        objetivo = "Gerar uma palavra aleatória (animal, objeto, etc). Nunca repita palavras."
    else:
        # Contador de histórias
        role = "Contador de Histórias"
        objetivo = "Continuar a história existente, incorporando a última palavra recebida."
    # Gera a resposta
    resposta = gerar_resposta(role, objetivo, mensagem_atual, memory, llm)
    print(f"{role}: {resposta}\n")
    mensagem_atual = resposta

The human mentions "Gato," prompting the AI to describe a scene at a birthday party where a curious cat captures everyone's attention. The cat, initially focused on a ball of yarn, becomes the center of amusement as it attempts to retrieve the ball from under the sofa, leading to laughter and joy among the guests. The human elaborates on the scene, detailing how the cat, with its bright and curious eyes, moves gracefully and becomes absorbed in playing with the yarn ball, oblivious to the party around it. When the ball rolls under the sofa, the cat's determined efforts to retrieve it draw the attention of the guests, who find the cat's antics hilarious, turning the party into a spectacle where the cat becomes the star of the night. The AI then simply responds with "Cachorro." The human mentions "Cachorro," and the AI describes how a golden retriever enters the room, curious about the commotion. The dog, wanting to join the fun, tries to help the cat retrieve the yarn ball by pushing th